In [ ]:
'''
2017年10月12日
    完成：
        对不同环境下的图片进行标定，用这些标定图片微调网络，能得到比较好的表现
        opencv读取视频得到的图片是BGR模式的，CUMT模型时RGB输入，需要调整
''

In [ ]:
import tensorflow as tf
import numpy as np
from cumt_eye import Cumt_itraker
import matplotlib.pyplot as plt
import os
from scipy.misc import imread
from scipy.misc import imresize
import UI
import cv2
sess=tf.InteractiveSession()
cumt_data=np.load(r'D:\Proj_DL\Code\Proj_EyeTraker\cumt_data.pkl')


X=tf.placeholder(shape=[None,32,128,3],dtype=tf.float32)
Y=tf.placeholder(shape=[None,10],dtype=tf.float32)
DROPOUT=tf.placeholder(dtype=tf.float32)
LR=tf.placeholder(dtype=tf.float32)
BN_FLAG=tf.placeholder(dtype=tf.bool)

with tf.name_scope('model'):
    model=Cumt_itraker(image=X,bn_flag=BN_FLAG,dropout_rate=DROPOUT)
    y_score=model.score

with tf.name_scope('LOSS'):
    LOSS=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_score,labels=Y))
    tf.summary.scalar('loss',LOSS)
with tf.name_scope('TRAIN'):
    TRAIN=tf.train.AdamOptimizer(LR).minimize(LOSS)
with tf.name_scope('ACCURACY'):
    acc_count=tf.equal(tf.arg_max(y_score,1),tf.arg_max(Y,1))
    ACCURACY=tf.reduce_mean(tf.cast(acc_count,tf.float32))
    tf.summary.scalar('acc',ACCURACY)

writer_tr=tf.summary.FileWriter('./mylog/test')
writer_te=tf.summary.FileWriter('./mylog/val')
merge=tf.summary.merge_all()
ex_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)



In [ ]:
#初始化变量，需要重新训练才运行
init=tf.global_variables_initializer()
sess.run(init)

In [ ]:
#从保存的检查点钟复原变量，加载原数据训练好的模型
saver=tf.train.Saver()
saver.restore(sess,r'D:\Proj_DL\Code\Proj_EyeTraker\model_save\model_71.ckpt-64600')


In [ ]:
#模拟校正的图片,训练图片为 RGB
d=np.load('./test_video/101603/101603_z.pkl')
val={}
mean_pic=np.mean(d['images'],0)
#mean_pic=[103.939, 116.779, 123.68]
val['images']=d['images'].astype('float32')-mean_pic
val['labels']=d['labels']
del d

In [ ]:
val['images'].shape

In [ ]:
#方法 1：随机抽取照片标定

tr_num=(val['images'].shape[0])*0.3#校正图片数目
tr_num=int(tr_num)
#修改，只用一部分视频数据来训练，看效果
all_picnum=val['images'].shape[0]
index=a=np.asarray(range(all_picnum))
np.random.shuffle(index)
tr_mask=index[:tr_num]
te_mask=index[tr_num:]
tr_img,tr_lab=val['images'][tr_mask],val['labels'][tr_mask]
te_img,te_lab=val['images'][te_mask],val['labels'][te_mask]


In [ ]:
tr_num

In [ ]:
saver=tf.train.Saver()

In [ ]:
# 150 loop/min
lr_=1e-3
for i in range(150*60*2):
    mask=np.random.choice(range(tr_num),64,replace=False)
    x_,y_=tr_img[mask],tr_lab[mask]
    #x_,y_=tr_img,tr_lab
    feed_dict={X:x_,Y:y_,DROPOUT:0.1,BN_FLAG:True,LR:lr_}
    sess.run([TRAIN,ex_ops],feed_dict=feed_dict)

    if i%10==0:
        x_,y_=te_img,te_lab
        feed_dict={X:x_,Y:y_,DROPOUT:1.,BN_FLAG:False,LR:lr_}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_te.add_summary(m_,i)
        if acc_>0.9999:
            m_name='./model_save/v3model_'+str(int(acc_*100))+'.ckpt'
            saver.save(sess,m_name,global_step=i)
            lr_=max(1e-5,0.5*lr_)
            print('Save model,epoch:{},validate accuracy:{},learning rate:{}'.format(i,acc_,lr_))
           
            break
        print('--epoch:{},loss:{},accuracy:{}'.format(i,loss_,acc_))

In [ ]:
#保存模型
saver=tf.train.Saver()
saver.save(sess,'./model_save/model.ckpt',global_step=1000)

In [ ]:
'''
在视频中插入判别网络
'''

In [ ]:
prob=tf.nn.softmax(y_score,name='prob')
y_guess=tf.arg_max(prob,1)+1

#预测方向到九宫格方向,
pre2block={2:3,3:2,4:1,
           5:6,6:5,7:4,
           8:9,9:8,10:7}
#校正时需要保存图像均值
VGG_MEAN =mean_pic

In [ ]:
#BGR
#VGG_MEAN = [103.939, 116.779, 123.68]


In [ ]:
cap = cv2.VideoCapture(r'test_video/101603.mp4')
fps=cap.get(cv2.CAP_PROP_FPS)
fsize=(
    int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
    int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
      )
# save video
video_saver=cv2.VideoWriter('demo_1041.flv',cv2.VideoWriter_fourcc('F','L','V','1'),50,fsize)
while (True):
    ret,frame = cap.read()
    
    if ret==True:
        #截取眼部图片
#         frame=np.transpose(frame,[1,0,2])
#         frame=frame[::-1,:,:]
#         eye_img=None
        #frame=frame[:,::-1,:]
        eye_img=UI.drew_face_eye(frame)
        if eye_img is None:
            #没有检测到眼睛，跳到下一帧
            cv2.imshow('frame',frame)
            video_saver.write(frame)
            if cv2.waitKey(10)& 0xff ==ord('q'):
                print('quite')
                break
            continue
        # BGR -> RGB
        eye_img=eye_img[:,:,::-1]
        e1=imresize(eye_img,(32,128,3)).astype('float32')-VGG_MEAN
        
        x_=e1.reshape((1,32,128,3))
        pre_asw=sess.run(y_guess,feed_dict={X:x_,DROPOUT:1.,BN_FLAG:False})[0]
        #给图片划线，划框
        print(pre_asw)
        frame=UI.drawline(frame,line_w=1)
        if pre_asw != 1:
            #有检测结果
            
            frame=UI.drawblock(frame,pre2block[pre_asw])
            #cv2.waitKey()
        #cv2.imshow('frame',x_.reshape((32,128,3)))
        cv2.imshow('frame',frame)
        video_saver.write(frame)
        if cv2.waitKey(10)& 0xff ==ord('q'):
            print('quite')
            break
    else:
        print('video end')
        break
cap.release()
video_saver.release()
cv2.destroyAllWindows()

In [ ]:
/**************************************************************************/
'''
用固定的眼部识别参数截取眼部图片
标注，保存
'''

In [ ]:
import cv2
import UI
import matplotlib.pyplot as plt
import os
import numpy as np
from scipy.misc import imread
from scipy.misc import imresize

#自然方向 -> 论文方向
key_map={
         1:'10',2:'9',3:'8',
         4:'7',5:'6',6:'5',
         7:'4',8:'3',9:'2',
         0:'1'
        }

In [ ]:
#改
cap = cv2.VideoCapture(r'test_video/101603.mp4')
i_counter=0
while (True):
    i_counter+=1
    ret,frame = cap.read()
    if ret==True:
        #截取眼部图片
        eye_img=UI.drew_face_eye(frame)
        if eye_img is not None:
            cv2.imshow('frame',eye_img)
            a=(cv2.waitKey())-48
            cv2.destroyWindow('frame')
            
            # Esc 退出，'.'表示放弃这张图
            if a==(-21):
                break
            elif a!=-2:
                resize_img=imresize(eye_img,(32,128,3)).astype('uint8')
                cv2.imwrite('./test_video/101603'+str(i_counter)+'_'+key_map[a]+'.jpg',resize_img)
    else:
        print('video end')
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
img_root='./test_video/101603'
lis=os.listdir(img_root)
img_arr,lab_arr=[],[]
for addr in lis:
    #标签
    lable_=[0]*10
    if addr[-5]==0:
        n=10
    else:
        n=int(addr[-5])
    lable_[n-1]=1
    lab_arr.append(lable_)
    #图像
    img_=imread(os.path.join(img_root,addr))
    #img_=imresize(img_,(32,128,3)).astype('uint8')
    img_arr.append(img_)
img_mat=np.array(img_arr)
lab_mat=np.array(lab_arr)

In [ ]:
img_mat.shape

In [ ]:
import pickle
fp=open('./test_video/101603/101603_z.pkl','wb')
pickle.dump(file=fp,obj={'images':img_mat,'labels':lab_mat})
fp.close()

In [ ]:
/********************************************************/
'''
视频分辨率转换
（M,N） -> (640,480)
'''

In [ ]:
import cv2
from scipy.misc import imresize
import numpy as np
ori_video=r'real-Z.mp4'
tar_video=r'real_h.flv'

In [ ]:
cap = cv2.VideoCapture(r'real_H.mp4')
fps=cap.get(cv2.CAP_PROP_FPS)
fsize=(
    int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
    int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
    
      )
# save video
video_saver=cv2.VideoWriter('real_h.flv',cv2.VideoWriter_fourcc('F','L','V','1'),30,fsize)
while (True):
    ret,frame = cap.read()
    if ret==True:
        frame=(np.transpose(frame,[1,0,2]))[::-1,:,:]
        cv2.imshow('frame',frame)
        video_saver.write(frame)
        if cv2.waitKey(10)& 0xff ==ord('q'):
            print('quite')
            break
    else:
        print('video end')
        break
cap.release()
video_saver.release()
cv2.destroyAllWindows()

In [ ]:
video_saver

In [ ]:
video_saver.release()